### MODELLING-
***
#### Here, we try to reduce the size of the data bu only considering the top 10 clusters adn applying our model on it.

In [13]:
import pandas as pd
import numpy as np
import pickle
import time
from datetime import date

import matplotlib.pyplot as plt
import seaborn as sns

import os
import gc
import zipfile
import io

In [14]:
from sklearn.model_selection import cross_val_score, GridSearchCV
from sklearn.model_selection import KFold,train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.pipeline import Pipeline

##### Read the cleaned dataset-

In [15]:
path_train_final ='F:/AML/Project/clean dataset/FinalExpedia_set_cleaned/Train_set_cleaned.csv'


train_set=pd.read_csv(path_train_final) 

In [16]:
train_set.shape

(2512931, 39)

##### Find out the top 10 clusters-

In [17]:
most_common_clusters = list(train_set.hotel_cluster.value_counts().head(10).index)
most_common_clusters

[91, 48, 42, 59, 28, 16, 18, 95, 50, 21]

In [18]:
train = train_set.loc[train_set['hotel_cluster'].isin(most_common_clusters)]

In [19]:
print("Length of new training dataset:", len(train))

Length of new training dataset: 568859


In [20]:
X = train.drop(['hotel_cluster'], axis=1)
y_target = train['hotel_cluster']
train = train[train['stay_dur']>0]
train = train[train['no_of_days_before_booking'] >0]
train = train.reset_index(drop=True)

In [21]:
X_train, X_test, y_train, y_test = train_test_split(X, y_target, test_size=0.2, random_state=1)

In [22]:
X_train.shape


(455087, 38)

#### Random Forest model-

In [23]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import KFold
from itertools import chain

all_probs = []
unique_clusters = train["hotel_cluster"].unique()
for cluster in unique_clusters:
    train["target"] = 1
    train["target"][train["hotel_cluster"] != cluster] = 0
    predictors = [col for col in train if col not in ["hotel_cluster", "target"]]
    probs = []
    cv = KFold(n_splits=10)
    clf = RandomForestClassifier(n_estimators=100, min_weight_fraction_leaf=0.1)
    for i, (tr, te) in  enumerate(cv.split(train["target"])):
        clf.fit(train[predictors].iloc[tr], train["target"].iloc[tr])
        preds = clf.predict_proba(train[predictors].iloc[te])
        probs.append([p[1] for p in preds])
    full_probs = chain.from_iterable(probs)
    all_probs.append(list(full_probs))

prediction_frame = pd.DataFrame(all_probs).T
prediction_frame.columns = unique_clusters
def find_top_5(row):
    return list(row.nlargest(5).index)

preds = []
for index, row in prediction_frame.iterrows():
    preds.append(find_top_5(row))

C:\Users\User-\AppData\Local\Temp\ipykernel_8244\787561592.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train["target"][train["hotel_cluster"] != cluster] = 0
C:\Users\User-\AppData\Local\Temp\ipykernel_8244\787561592.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train["target"][train["hotel_cluster"] != cluster] = 0
C:\Users\User-\AppData\Local\Temp\ipykernel_8244\787561592.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-cop

In [24]:
import numpy as np

def apk(actual, predicted, k=10):
    """
    Computes the average precision at k.

    This function computes the average prescision at k between two lists of
    items.

    Parameters
    ----------
    actual : list
             A list of elements that are to be predicted (order doesn't matter)
    predicted : list
                A list of predicted elements (order does matter)
    k : int, optional
        The maximum number of predicted elements

    Returns
    -------
    score : double
            The average precision at k over the input lists

    """
    if len(predicted)>k:
        predicted = predicted[:k]

    score = 0.0
    num_hits = 0.0

    for i,p in enumerate(predicted):
        if p in actual and p not in predicted[:i]:
            num_hits += 1.0
            score += num_hits / (i+1.0)

    if not actual:
        return 0.0

    return score / min(len(actual), k)

def mapk(actual, predicted, k=10):
    """
    Computes the mean average precision at k.

    This function computes the mean average prescision at k between two lists
    of lists of items.

    Parameters
    ----------
    actual : list
             A list of lists of elements that are to be predicted 
             (order doesn't matter in the lists)
    predicted : list
                A list of lists of predicted elements
                (order matters in the lists)
    k : int, optional
        The maximum number of predicted elements

    Returns
    -------
    score : double
            The mean average precision at k over the input lists

    """
    return np.mean([apk(a,p,k) for a,p in zip(actual, predicted)])

In [25]:
mapk([[l] for l in train["hotel_cluster"]], preds, k=5)

0.4219242957354769